
# Generating Bash Code with Granite Code and Ollama


### Prerequisite: Install Ollama and Granite Code models

1. [Download and Start Ollama](https://ollama.com/download)
1. Install Granite Code 20b: `ollama pull granite-code:20b`
1. Install Granite Code 8b: `ollama pull granite-code:8b`

In [ ]:
!pip install ollama


## One-shot Prompt with Granite Code 20b

In One-shot prompting, you provide the model with a question and no examples. The model will generate an answer given its training. Larger models tend to do better at this task.

Use the [ollama-python package](https://github.com/ollama/ollama-python) to access the model.

In [ ]:
import ollama

response = ollama.generate(
  model='granite-code:20b',
  prompt="How can I use a bash script to calculate the number of days between the dates Jan 12, 2023 and Jan 20, 2024?"
)
print(response["response"])

If you are running on a Mac, it is likely that the invocation of `date` will be incorrect for the MacOS version of this command. Try adding the following sentence to the prompt and see what happens: `Make sure the generated shell commands are MacOS-compatible.`

## Few-shot Prompting with Granite Code 8b

In few-shot prompting, you provide the model with a question and some examples. The model will generate an answer given its training. The additional examples help the model zero in on a pattern, which may be required for smaller models to perform well at this task.

In [ ]:
examples = """
Question:
Recursively finds files like '*.js', and filters out files with 'excludeddir' in path.
Answer:
find . -name '*.js' | grep -v excludeddir

Question:
Dump \"a0b\" as hexadecimal bytes
Answer:
printf \"a0b\" | od -tx1

Question:
create a tar ball of all pdf files in current folder
Answer:
find . -name *.pdf | xargs tar czvf /root/Desktop/evidence/pdf.tar

Question:
Sort all files/directories under current directory according to modification time and print only the recent 7 of them
Answer:
find -mindepth 1 -printf \"%T@ %Pn\" | sort -n -r | cut -d' ' -f 2- | tail -n +7

Question:
find all the empty directories in the current folder
Answer:
find . -type d -empty

"""

query = "How can I use a bash script to calculate the number of days between the dates Jan 12, 2023 and Jan 20, 2024?"

user_prompt = examples + "\nQuestion:\n" + query + "\nAnswer:\n"

response = ollama.chat(model='granite-code:8b', messages=[
  {
    'role': 'user',
    'content': user_prompt
  },
])
print(response['message']['content'])

## Adding a System Prompt

A system prompt can be used to provide additional instructions and clarity or context for the task. Here we let the model know what we expect from it.

In [ ]:
system_prompt = "You are a helpful software engineer. You write clear, concise, well-commented code."

response = ollama.chat(model='granite-code:8b', messages=[
  {
    'role':'system',
    'content': system_prompt  
  },
  {
    'role': 'user',
    'content': user_prompt
  },
])
print(response['message']['content'])
